# Z-Image & Z-Image Turbo

**Module 7.4, Lesson 4** | CourseAI

Z-Image takes the same building blocks you know -- transformers, patchify, flow matching, joint attention, adaLN -- and recombines them more efficiently. Its S3-DiT single-stream architecture eliminates the parameter overhead of MMDiT's dual-stream design. A single LLM (Qwen3-4B) replaces triple text encoders. And Z-Image Turbo generates images in 8 steps, fitting in under 16GB of VRAM.

**What you will do:**
- Load Z-Image Turbo and inspect the architecture: verify the single-stream transformer, the Qwen3-4B text encoder, and parameter counts
- Extract the transformer block structure and verify shared Q/K/V projections -- no dual-stream split, no per-modality duplication
- Generate images with Z-Image Turbo at different step counts, test bilingual capability (English and Chinese prompts), and measure inference time
- Compare Z-Image's architecture and outputs against SD3 or Flux, tracing the full pipeline end-to-end

**For each exercise, PREDICT the output before running the cell.**

Every concept in this notebook comes from the lesson. S3-DiT as "translate once, then speak together," Qwen3-4B as the endpoint of the text encoder evolution, single-stream as simplification not novelty. No new theory -- just hands-on verification of what you just read.

**Estimated time:** 60-90 minutes. All exercises require a GPU runtime with at least 16 GB VRAM (Colab T4 works, A100 is comfortable). Z-Image Turbo fits in under 16GB in bfloat16.

## Setup

Run this cell to install dependencies and configure the environment.

**Important:** Switch to a GPU runtime in Colab (Runtime > Change runtime type > T4 GPU or better). Z-Image Turbo requires ~12-14 GB VRAM in bfloat16. A T4 (16 GB) works; an A100 (40 GB) is comfortable.

In [ ]:
# Install diffusers from source for Z-Image support
# Z-Image was merged into diffusers v0.36.0+
!pip install -q git+https://github.com/huggingface/diffusers
!pip install -q transformers accelerate safetensors sentencepiece protobuf

In [ ]:
import torch
import torch.nn as nn
import gc
import time
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import display

# Reproducible results
torch.manual_seed(42)

# Device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
dtype = torch.bfloat16 if torch.cuda.is_available() else torch.float32

# Nice plots
plt.style.use('dark_background')
plt.rcParams['figure.figsize'] = [14, 5]
plt.rcParams['figure.dpi'] = 100

print(f'Device: {device}')
print(f'Dtype: {dtype}')
if device.type == 'cpu':
    print('WARNING: No GPU detected. All exercises in this notebook require a GPU.')
    print('Switch to GPU: Runtime > Change runtime type > T4 GPU')
print()
print('Setup complete.')

## Shared Helpers

Utility functions used across multiple exercises. Run this cell now.

In [ ]:
def count_parameters(model):
    """Count total parameters in a model."""
    return sum(p.numel() for p in model.parameters())


def count_parameters_by_type(model):
    """Count parameters grouped by module type (e.g., Linear, LayerNorm)."""
    counts = {}
    for name, module in model.named_modules():
        module_type = module.__class__.__name__
        if module_type == type(model).__name__:
            continue
        n_params = sum(p.numel() for p in module.parameters(recurse=False))
        if n_params > 0:
            counts[module_type] = counts.get(module_type, 0) + n_params
    return dict(sorted(counts.items(), key=lambda x: -x[1]))


def show_image_row(images, titles, suptitle=None, figsize=None):
    """Display a row of PIL images with titles."""
    n = len(images)
    fig_w = figsize[0] if figsize else max(5 * n, 12)
    fig_h = figsize[1] if figsize else 5
    fig, axes = plt.subplots(1, n, figsize=(fig_w, fig_h))
    if n == 1:
        axes = [axes]
    for ax, img, title in zip(axes, images, titles):
        ax.imshow(img)
        ax.set_title(title, fontsize=10)
        ax.axis('off')
    if suptitle:
        plt.suptitle(suptitle, fontsize=13, y=1.02)
    plt.tight_layout()
    plt.show()


def free_memory(*objects):
    """Delete objects and free GPU memory."""
    for obj in objects:
        del obj
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    print('Memory freed.')


print('Helpers defined: count_parameters, count_parameters_by_type, show_image_row, free_memory')

---

## Exercise 1: Z-Image Pipeline Inspection `[Guided]`

The lesson taught that Z-Image uses a **single LLM text encoder** (Qwen3-4B) instead of SD3's triple encoder setup, and an **S3-DiT single-stream transformer** instead of MMDiT's dual-stream design. The result: 6.15B parameters competing with Flux's 32B.

In this exercise, you will load Z-Image Turbo and verify all of this concretely:
1. Print the text encoder class name and parameter count -- verify it is Qwen3-4B (~4B params)
2. Verify embedding output shapes from the text encoder
3. Inspect the S3-DiT transformer: count parameters, verify the single-stream architecture
4. Compare the overall parameter budget to SD3 and Flux

**Before running, predict:**
- Z-Image replaced SD3's three text encoders (~5.2B combined) with one LLM (Qwen3-4B). How many parameters does the text encoder have?
- The lesson showed the S3-DiT has 30 layers with d_model=3840. How many total transformer parameters would you estimate?
- SD3 had separate Q/K/V projections per modality. Z-Image has shared projections. What structural difference should you see when inspecting a transformer block?

In [ ]:
# ============================================================
# Exercise 1: Load Z-Image Turbo and inspect its components
# ============================================================

# --- Step 1: Load Z-Image Turbo ---
# Z-Image Turbo is available via the diffusers ZImagePipeline.
# We load in bfloat16 for optimal memory efficiency.
# The model fits in <16GB VRAM.

from diffusers import ZImagePipeline

print('Loading Z-Image Turbo... (this may take a few minutes)')
pipe = ZImagePipeline.from_pretrained(
    'Tongyi-MAI/Z-Image-Turbo',
    torch_dtype=torch.bfloat16,
)
pipe = pipe.to(device)
print('Z-Image Turbo loaded.')
print()

In [ ]:
# --- Step 2: Inspect the text encoder ---
# Z-Image uses a SINGLE text encoder: Qwen3-4B (a chat-capable LLM).
# Compare this to SD3's triple encoder setup:
#   SD3: CLIP ViT-L (123M) + OpenCLIP ViT-bigG (354M) + T5-XXL (4.7B) = ~5.2B total
#   Z-Image: Qwen3-4B (~4B) = one encoder

text_encoder = pipe.text_encoder
encoder_params = count_parameters(text_encoder)

print('Z-Image Text Encoder:')
print('=' * 60)
print(f'  Class: {type(text_encoder).__name__}')
print(f'  Parameters: {encoder_params:,} ({encoder_params / 1e9:.2f}B)')
print()

# Compare to SD3's text encoder setup
print('Text Encoder Comparison:')
print('-' * 60)
print(f'  SD v1.5:   CLIP ViT-L              ~0.12B (1 encoder)')
print(f'  SDXL:      CLIP ViT-L + OpenCLIP    ~0.48B (2 encoders)')
print(f'  SD3/Flux:  CLIP + OpenCLIP + T5-XXL ~5.2B  (3 encoders)')
print(f'  Z-Image:   Qwen3-4B                 ~{encoder_params / 1e9:.1f}B  (1 encoder)')
print()
print('The trajectory: from multiple specialized encoders back to one')
print('powerful encoder. Simpler pipeline, richer embeddings.')

In [ ]:
# --- Step 3: Verify embedding output shapes ---
# Encode a test prompt through Qwen3-4B and check the output shape.
# The LLM produces per-token embeddings that enter the S3-DiT
# as text tokens in the unified sequence.

test_prompt = 'a cat sitting on a beach at sunset'

# Tokenize and encode
tokenizer = pipe.tokenizer
tokens = tokenizer(
    test_prompt,
    return_tensors='pt',
    padding='max_length',
    max_length=512,
    truncation=True,
).input_ids.to(device)

print(f'Prompt: "{test_prompt}"')
print(f'Tokenized shape: {list(tokens.shape)}')
print(f'Max sequence length: 512')
print()

# Get embeddings from the text encoder
with torch.no_grad():
    text_outputs = text_encoder(tokens, output_hidden_states=True)

# The final hidden state provides the per-token embeddings
hidden_states = text_outputs.last_hidden_state
print(f'Text encoder output shape: {list(hidden_states.shape)}')
print(f'  Batch size: {hidden_states.shape[0]}')
print(f'  Sequence length: {hidden_states.shape[1]}')
print(f'  Embedding dimension: {hidden_states.shape[2]}')
print()
print('These per-token embeddings enter the S3-DiT as text tokens.')
print('The LLM processes the prompt ONCE to produce embeddings --')
print('no chat, no reasoning at inference time. Just rich embeddings')
print('from a model trained on diverse language tasks.')

In [ ]:
# --- Step 4: Inspect the S3-DiT transformer ---
# The denoising network is pipe.transformer (a ZImageTransformer2DModel).
# The lesson predicted: 30 layers, d_model=3840, 30 heads, single-stream.

transformer = pipe.transformer
transformer_params = count_parameters(transformer)

print('S3-DiT Transformer (Denoising Network):')
print('=' * 60)
print(f'  Class: {type(transformer).__name__}')
print(f'  Parameters: {transformer_params:,} ({transformer_params / 1e9:.2f}B)')
print()

# Print config values
config = transformer.config
print('Config:')
for key in ['num_layers', 'num_attention_heads', 'attention_head_dim',
            'in_channels', 'patch_size', 'joint_attention_dim']:
    val = getattr(config, key, 'N/A')
    print(f'  {key}: {val}')
print()

# Parameter breakdown by top-level component
print('Parameter breakdown by top-level component:')
groups = {}
for name, module in transformer.named_children():
    n_params = count_parameters(module)
    if n_params > 0:
        groups[name] = n_params

for group, count in sorted(groups.items(), key=lambda x: -x[1]):
    pct = count / transformer_params * 100
    print(f'    {group:<35} {count:>14,} ({pct:.1f}%)')

In [ ]:
# --- Step 5: Overall parameter budget ---
# Compare Z-Image's total parameter count to SD3 and Flux.

vae_params = count_parameters(pipe.vae)
total_params = encoder_params + transformer_params + vae_params

print('Z-Image Turbo: Parameter Budget')
print('=' * 60)
print(f'  Text encoder (Qwen3-4B):   {encoder_params:>14,} ({encoder_params / 1e9:.2f}B)')
print(f'  S3-DiT transformer:        {transformer_params:>14,} ({transformer_params / 1e9:.2f}B)')
print(f'  VAE:                       {vae_params:>14,} ({vae_params / 1e6:.0f}M)')
print('-' * 60)
print(f'  Total:                     {total_params:>14,} ({total_params / 1e9:.2f}B)')
print()

print('Comparison to other models:')
print('-' * 60)
print(f'  SD3 Medium:    ~5.2B (text) + ~2B (MMDiT)  = ~7.2B total')
print(f'  Flux.1 Dev:    ~5.2B (text) + ~12B (MMDiT) = ~32B total')
print(f'  Z-Image:       ~{encoder_params / 1e9:.1f}B (text) + ~{transformer_params / 1e9:.1f}B (S3-DiT) = ~{total_params / 1e9:.1f}B total')
print()
print(f'Z-Image is roughly {32 / (total_params / 1e9):.0f}x smaller than Flux.1 Dev.')
print('The savings come from two sources:')
print('  1. One text encoder instead of three (simpler pipeline)')
print('  2. Single-stream transformer (no per-modality parameter duplication)')

### What Just Happened

You loaded Z-Image Turbo and verified the architecture the lesson described:

- **One text encoder, not three.** Qwen3-4B (~4B params) replaces the CLIP ViT-L + OpenCLIP ViT-bigG + T5-XXL triple encoder setup from SD3. One tokenizer, one forward pass, one set of embeddings. The text encoder evolution: 1 encoder -> 2 -> 3 -> back to 1 (but a much more powerful 1).

- **S3-DiT is a standard transformer.** The denoising network is a `ZImageTransformer2DModel` -- a transformer with 30 layers, 3840 hidden dimension, 30 attention heads. The same scaling recipe from the DiT lesson ("two knobs: d_model and N"), applied at a larger scale.

- **Dramatically fewer parameters than Flux.** Z-Image achieves comparable quality with roughly 1/5 the parameters. The savings come from architectural simplification (single-stream) and a simpler text encoding pipeline (one LLM instead of three specialized encoders).

- **The LLM provides embeddings, not chat.** Qwen3-4B processes the prompt once to produce token embeddings. No interactive reasoning at inference time -- just rich embeddings from a model trained on diverse language tasks.

---

## Exercise 2: Single-Stream Architecture Exploration `[Guided]`

The lesson explained the key architectural difference between MMDiT and S3-DiT:

- **MMDiT (SD3/Flux):** Separate Q/K/V projections and separate FFN for text and image tokens at every layer. "Shared listening, separate thinking."
- **S3-DiT (Z-Image):** Shared Q/K/V projections and shared FFN for all token types. Modality-specific processing concentrated in 2 lightweight refiner layers upfront. "Translate once, then speak together."

In this exercise, you will:
1. Extract the transformer block structure from Z-Image
2. Verify shared Q/K/V projections (no modality-specific duplication)
3. Identify the refiner layers
4. Count parameter savings vs what a dual-stream equivalent would need

**Before running, predict:**
- In MMDiT, each block has separate text and image Q/K/V projections. In S3-DiT, you should see ONE set of Q/K/V projections. What does this look like in the module listing?
- The lesson claimed single-stream saves ~50% of projection + FFN parameters per block. For d_model=3840, how many parameters does one set of Q/K/V projections cost? (Hint: 3 matrices of d_model x d_model)
- Where will you find the refiner layers -- inside each transformer block, or as a separate module?

In [ ]:
# ============================================================
# Exercise 2: Inspect the single-stream block structure
# ============================================================

# --- Step 1: List all transformer blocks ---
# The transformer has a list of blocks. In MMDiT, these would have
# separate per-modality components. In S3-DiT, they should be
# single-stream: one set of projections, one FFN.

transformer = pipe.transformer

# Count transformer blocks
print('Transformer block structure:')
print('=' * 60)

# List all top-level children
for name, module in transformer.named_children():
    n_params = count_parameters(module)
    if n_params > 0:
        # Check if it's a list of blocks
        if hasattr(module, '__len__'):
            print(f'  {name}: {type(module).__name__} ({len(module)} items, {n_params:,} total params)')
        else:
            print(f'  {name}: {type(module).__name__} ({n_params:,} params)')

In [ ]:
# --- Step 2: Inspect one S3-DiT block ---
# The key architectural claim: ONE set of Q/K/V projections shared
# across all token types (text, image, visual semantic).
# Compare this to MMDiT where you would see SEPARATE projections.

# Get the blocks -- look for the main transformer blocks
blocks = None
block_container_name = None
for name, module in transformer.named_children():
    if hasattr(module, '__len__') and len(module) > 10:
        blocks = module
        block_container_name = name
        break

if blocks is None:
    # Fallback: try common names
    for attr in ['transformer_blocks', 'blocks', 'layers']:
        if hasattr(transformer, attr):
            blocks = getattr(transformer, attr)
            block_container_name = attr
            break

print(f'Main transformer blocks: {block_container_name}')
print(f'Number of blocks: {len(blocks)}')
print()

# Inspect block 0 in detail
block_0 = blocks[0]
print(f'Block 0 type: {type(block_0).__name__}')
print(f'Block 0 total params: {count_parameters(block_0):,}')
print()
print('Sub-modules (looking for SHARED projections, not per-modality):')
print('-' * 80)
for name, module in block_0.named_modules():
    if name == '':
        continue
    n_params = sum(p.numel() for p in module.parameters(recurse=False))
    if n_params > 0:
        print(f'  {name:<55} {type(module).__name__:<15} ({n_params:,})')

In [ ]:
# --- Step 3: Identify attention projections ---
# In MMDiT, you would see:
#   attn.to_q, attn.to_k, attn.to_v         (image projections)
#   attn.add_q_proj, attn.add_k_proj, ...    (text projections)
# In S3-DiT single-stream, you should see:
#   ONE set of Q/K/V projections, shared across all modalities

print('Attention module analysis:')
print('=' * 60)

# Find the attention module
attn_module = None
attn_name = None
for name, module in block_0.named_modules():
    mod_type = type(module).__name__
    if 'Attention' in mod_type or 'attn' in name.lower():
        if sum(p.numel() for p in module.parameters(recurse=False)) > 0 or \
           sum(1 for _ in module.children()) > 0:
            attn_module = module
            attn_name = name
            break

if attn_module is None:
    # Try direct attribute access
    for attr in ['attn', 'attn1', 'self_attn', 'attention']:
        if hasattr(block_0, attr):
            attn_module = getattr(block_0, attr)
            attn_name = attr
            break

print(f'Attention module: {attn_name} ({type(attn_module).__name__})')
print(f'Attention params: {count_parameters(attn_module):,}')
print()

# List all children of the attention module
print('Attention sub-modules:')
for name, child in attn_module.named_children():
    n_params = count_parameters(child)
    print(f'  {name:<30} {type(child).__name__:<15} ({n_params:,})')

print()
print('Key observation: Look for the Q/K/V projections.')
print('In MMDiT you would see DUPLICATE sets (one per modality).')
print('In S3-DiT you should see ONE set (shared across modalities).')
print('This is the "translate once, then speak together" design.')

In [ ]:
# --- Step 4: Identify refiner layers ---
# The lesson taught that S3-DiT uses 2 lightweight refiner layers
# per modality to pre-process embeddings before the main transformer.
# These should be separate from the main 30-layer backbone.

print('Looking for refiner layers:')
print('=' * 60)

# Search for refiner-related modules in the transformer
refiner_found = False
for name, module in transformer.named_children():
    name_lower = name.lower()
    if 'refin' in name_lower or 'pre' in name_lower or 'embed' in name_lower:
        n_params = count_parameters(module)
        if n_params > 0:
            print(f'  {name}: {type(module).__name__} ({n_params:,} params)')
            refiner_found = True

if not refiner_found:
    print('  No obviously named refiner layers found at top level.')
    print('  Searching all named modules for refiner-like components...')
    for name, module in transformer.named_modules():
        if 'refin' in name.lower():
            n_params = sum(p.numel() for p in module.parameters(recurse=False))
            print(f'  {name}: {type(module).__name__} ({n_params:,})')
            refiner_found = True

if not refiner_found:
    print('  Refiner layers may be integrated into the embedding or')
    print('  projection stages. Check the input projection modules above.')

print()
print('The refiner layers serve a critical role: they pre-process each')
print("modality's raw embeddings into a shared representation space.")
print('Without them, shared Q/K/V projections would fail -- the same')
print('problem MMDiT solved with per-modality projections at every layer.')
print('S3-DiT solves it ONCE upfront instead of at every layer.')

In [ ]:
# --- Step 5: Parameter savings calculation ---
# The lesson claimed single-stream saves ~50% of projection + FFN
# parameters per block. Let's verify with real numbers.

d_model = None
# Try to get d_model from config
for attr in ['joint_attention_dim', 'hidden_size', 'inner_dim']:
    val = getattr(transformer.config, attr, None)
    if val is not None:
        d_model = val
        break

# Fallback: compute from attention heads
if d_model is None:
    n_heads = transformer.config.num_attention_heads
    head_dim = transformer.config.attention_head_dim
    d_model = n_heads * head_dim

n_blocks = len(blocks)
block_params = count_parameters(block_0)

print('Parameter Savings: Single-Stream vs Dual-Stream')
print('=' * 60)
print(f'  d_model: {d_model}')
print(f'  Number of blocks: {n_blocks}')
print(f'  Actual params per S3-DiT block: {block_params:,}')
print()

# Calculate what a dual-stream equivalent would need
# MMDiT dual-stream adds: separate Q/K/V for second modality + separate FFN
qkv_params = 3 * d_model * d_model  # Q, K, V matrices for one modality
# SwiGLU FFN: gate_proj (d * 8/3*d) + up_proj (d * 8/3*d) + down_proj (8/3*d * d)
ffn_dim = int(d_model * 8 / 3)  # Approximate SwiGLU hidden dim
ffn_params = 3 * d_model * ffn_dim  # gate + up + down projections

extra_dual_stream = qkv_params + ffn_params  # Per-modality duplication cost

print(f'Per-modality duplication cost (if dual-stream):')
print(f'  Extra Q/K/V projections:  {qkv_params:,} ({qkv_params / 1e6:.1f}M)')
print(f'  Extra FFN:               {ffn_params:,} ({ffn_params / 1e6:.1f}M)')
print(f'  Total extra per block:   {extra_dual_stream:,} ({extra_dual_stream / 1e6:.1f}M)')
print(f'  Over {n_blocks} blocks:          {extra_dual_stream * n_blocks:,} ({extra_dual_stream * n_blocks / 1e9:.2f}B)')
print()

savings_pct = extra_dual_stream / (block_params + extra_dual_stream) * 100
print(f'Parameter savings from single-stream:')
print(f'  Dual-stream block would be: {block_params + extra_dual_stream:,}')
print(f'  Single-stream block is:     {block_params:,}')
print(f'  Savings per block:          {savings_pct:.0f}% of projection + FFN params')
print(f'  Total savings:              {extra_dual_stream * n_blocks / 1e9:.2f}B parameters')
print()
print('This confirms the lesson: single-stream saves ~50% of per-block')
print('parameters by eliminating modality-specific duplication.')
print('Not from a novel mechanism, but from eliminating redundancy.')

### What Just Happened

You explored Z-Image's single-stream architecture and verified the design choices from the lesson:

- **Shared Q/K/V projections confirmed.** Unlike MMDiT's separate per-modality projections, each S3-DiT block has ONE set of Q/K/V projections. Text tokens, image tokens, and visual semantic tokens all share the same projections. This is the architectural source of the parameter savings.

- **Shared FFN confirmed.** One SwiGLU FFN per block instead of separate FFNs per modality. Same story: shared computation across all token types.

- **Refiner layers handle modality pre-processing.** The refiner layers (or equivalent input projection modules) pre-process each modality's raw embeddings into a shared representation space. This solves the same problem MMDiT solved with per-modality projections at every layer -- but solves it once upfront instead of 30 times.

- **~50% parameter savings verified.** A dual-stream equivalent of Z-Image's transformer would require roughly double the projection and FFN parameters per block. Over 30 layers, that adds up to billions of parameters. Single-stream is not a trick -- it is a fundamental architectural simplification.

- **The "translate once, then speak together" analogy holds.** MMDiT translates at every layer (separate projections). S3-DiT translates once (refiner layers), then everyone shares the same projections and FFN. Same communication, less overhead.

---

## Exercise 3: Z-Image Turbo Generation `[Supported]`

The lesson taught that Z-Image Turbo generates images in 8 steps via Decoupled-DMD distillation, fitting in under 16GB of VRAM. The distilled model uses `guidance_scale=0.0` (no classifier-free guidance needed -- the distillation bakes in the quality boost).

In this exercise, you will:
1. Generate images with Z-Image Turbo at different step counts (4, 8, 16 steps)
2. Compare quality across step counts to observe the Decoupled-DMD sweet spot
3. Test bilingual capability (English and Chinese prompts)
4. Measure inference time

Fill in the TODO markers to complete the comparison.

**Important note:** Z-Image Turbo uses `num_inference_steps=N+1` to produce N actual DiT forward passes. So `num_inference_steps=9` gives 8 steps. Also, guidance scale should be 0.0 for the Turbo variant.

In [ ]:
# ============================================================
# Exercise 3: Generate with Z-Image Turbo
# ============================================================

# --- Step 1: Generate at different step counts ---
# The Decoupled-DMD distillation means Z-Image Turbo produces
# good results at 8 steps. Let's compare 4, 8, and 16 steps.
#
# IMPORTANT: Z-Image Turbo uses num_inference_steps = N+1 for N actual
# DiT forward passes. So for 8 steps, use num_inference_steps=9.
# Also: guidance_scale=0.0 for Turbo models (no CFG needed).

prompt = 'a cat sitting on a beach at sunset, photorealistic, golden hour lighting'
height, width = 1024, 1024

# Map desired steps to num_inference_steps (N+1 pattern)
step_configs = [
    (4, 5),   # 4 actual steps -> num_inference_steps=5
    (8, 9),   # 8 actual steps -> num_inference_steps=9
    (16, 17), # 16 actual steps -> num_inference_steps=17
]

images = []
titles = []

for actual_steps, inference_steps in step_configs:
    generator = torch.Generator(device='cuda').manual_seed(42)
    start = time.time()

    # TODO: Generate an image using pipe() with these parameters:
    #   prompt=prompt
    #   height=height, width=width
    #   num_inference_steps=inference_steps
    #   guidance_scale=0.0  (important: 0 for Turbo models)
    #   generator=generator
    # Store the result in a variable called 'result'.
    raise NotImplementedError(
        'TODO: Call pipe() with the correct arguments.'
    )

    elapsed = time.time() - start
    images.append(result.images[0])
    titles.append(f'{actual_steps} steps\n{elapsed:.1f}s')
    print(f'  {actual_steps} steps ({inference_steps} inference): {elapsed:.1f}s')

print('\nGeneration complete.')

In [ ]:
# --- Step 2: Display the step count comparison ---

show_image_row(
    images, titles,
    suptitle='Z-Image Turbo: Quality vs Step Count (Decoupled-DMD)',
    figsize=(18, 6),
)

print('What you should observe:')
print('  4 steps:  Reasonable quality, may lack fine detail')
print('  8 steps:  The Decoupled-DMD sweet spot -- high quality')
print('  16 steps: Marginal improvement over 8 steps')
print()
print('Compare to the speed landscape from the lesson:')
print('  DDPM:         50+ steps  (baseline)')
print('  DDIM:         20-50 steps')
print('  LCM:          4-8 steps  (noticeable quality loss)')
print('  SDXL Turbo:   1-4 steps  (good quality, limited diversity)')
print('  Z-Image Turbo: 8 steps   (competitive with many-step models)')

In [ ]:
# --- Step 3: Test bilingual capability ---
# Z-Image uses Qwen3-4B, which is bilingual (English and Chinese).
# The lesson taught that Qwen3-4B provides richer text understanding
# including native bilingual support.

bilingual_prompts = [
    ('A majestic mountain landscape at sunrise with a river flowing through a valley',
     'English'),
    ('\u65e5\u51fa\u65f6\u5206\uff0c\u4e00\u6761\u6cb3\u6d41\u7a7f\u8fc7\u5c71\u8c37\uff0c\u58ee\u4e3d\u7684\u5c71\u8109\u666f\u89c2',
     'Chinese (same meaning)'),
]

bi_images = []
bi_titles = []

for prompt_text, label in bilingual_prompts:
    generator = torch.Generator(device='cuda').manual_seed(42)

    # TODO: Generate an image with 8 steps (num_inference_steps=9).
    # Use guidance_scale=0.0 and the prompt_text variable.
    raise NotImplementedError(
        'TODO: Generate an image for each bilingual prompt.'
    )

    bi_images.append(result.images[0])
    bi_titles.append(f'{label}\n"{prompt_text[:50]}..."' if len(prompt_text) > 50 else f'{label}\n"{prompt_text}"')

show_image_row(
    bi_images, bi_titles,
    suptitle='Z-Image Turbo: Bilingual Capability (English vs Chinese)',
    figsize=(14, 6),
)

print('Qwen3-4B is natively bilingual (English + Chinese).')
print('Both prompts describe the same scene. The images should be')
print('semantically similar, demonstrating that the LLM text encoder')
print('understands both languages equally well.')
print()
print('CLIP-based models would need a separate Chinese CLIP or')
print('translation step. Z-Image handles this natively because')
print('the text encoder is a chat-capable LLM trained on diverse data.')

In [ ]:
# --- Step 4: Measure inference time ---
# Time the full pipeline for 8 steps, averaged over 3 runs.
# This includes text encoding, denoising, and VAE decode.

prompt = 'a futuristic city skyline at night with neon lights'
n_runs = 3
times = []

# Warmup run
generator = torch.Generator(device='cuda').manual_seed(0)
_ = pipe(
    prompt=prompt, height=1024, width=1024,
    num_inference_steps=9, guidance_scale=0.0,
    generator=generator,
)

# Timed runs
for i in range(n_runs):
    generator = torch.Generator(device='cuda').manual_seed(i)
    if torch.cuda.is_available():
        torch.cuda.synchronize()
    start = time.time()

    _ = pipe(
        prompt=prompt, height=1024, width=1024,
        num_inference_steps=9, guidance_scale=0.0,
        generator=generator,
    )

    if torch.cuda.is_available():
        torch.cuda.synchronize()
    elapsed = time.time() - start
    times.append(elapsed)
    print(f'  Run {i+1}: {elapsed:.2f}s')

avg_time = sum(times) / len(times)
print(f'\nAverage generation time (8 steps, 1024x1024): {avg_time:.2f}s')
print(f'Steps per second: {8 / avg_time:.1f}')
print()
print('The lesson reported sub-second on H800 GPUs.')
print('On Colab GPUs (T4/A100), expect 2-10 seconds depending on the GPU.')
print('The key point: 8 steps is dramatically fewer than DDPM\'s 50+.')

<details>
<summary>Solution</summary>

The key insights: Z-Image Turbo uses `guidance_scale=0.0` because the Decoupled-DMD distillation has already baked in the quality boost that CFG provides. And `num_inference_steps` must be set to N+1 for N actual DiT forward passes.

**Step 1: Generate at different step counts**
```python
result = pipe(
    prompt=prompt,
    height=height,
    width=width,
    num_inference_steps=inference_steps,
    guidance_scale=0.0,
    generator=generator,
)
```

**Step 3: Bilingual generation**
```python
result = pipe(
    prompt=prompt_text,
    height=1024,
    width=1024,
    num_inference_steps=9,
    guidance_scale=0.0,
    generator=generator,
)
```

**What to observe:**
- At 8 steps, Z-Image Turbo produces high-quality photorealistic images. This is the sweet spot from Decoupled-DMD distillation.
- The jump from 4 to 8 steps is more significant than from 8 to 16. Decoupled-DMD was optimized for ~8 steps.
- English and Chinese prompts produce semantically similar images, demonstrating Qwen3-4B's bilingual capability. CLIP-based models would struggle with Chinese text without additional translation.
- Inference time on Colab should be a few seconds. On enterprise H800 GPUs, Z-Image Turbo achieves sub-second generation.

**Why guidance_scale=0.0?**
The Decoupled-DMD distillation process uses CFG-augmented teacher outputs (the "spear") during training. The quality boost from CFG is baked into the student model's weights. At inference time, no additional CFG is needed -- the model generates high-quality images directly. This is why the Turbo variant is faster: no need for the unconditional pass that CFG requires.

**Common mistakes:**
- Using `guidance_scale > 0` with the Turbo model. This can degrade quality because the model was not trained to work with inference-time CFG.
- Using `num_inference_steps=8` instead of 9. The pipeline uses N+1 steps internally.
- Forgetting to set the generator seed, making results non-reproducible across step counts.

</details>

### What Just Happened

You generated images with Z-Image Turbo and verified the lesson's claims:

- **8 steps is the sweet spot.** The Decoupled-DMD distillation was optimized for 8 steps. Quality at 8 steps is competitive with many-step models. The jump from 4 to 8 is significant; from 8 to 16 is marginal.

- **No guidance needed.** `guidance_scale=0.0` because the distillation baked in the quality boost. The "spear" (CFG-augmented teacher outputs) was used during training, so the student model generates high-quality images directly. No unconditional pass needed at inference.

- **Bilingual generation works natively.** English and Chinese prompts produce semantically similar images because Qwen3-4B is natively bilingual. No translation step, no separate encoder. This is a practical benefit of using an LLM as the text encoder.

- **Fast inference.** Sub-second on enterprise GPUs, a few seconds on Colab GPUs. 8 steps instead of DDPM's 50+. The combination of fewer steps (Decoupled-DMD) and a smaller model (single-stream) makes Z-Image Turbo practical for real-time applications.

---

## Exercise 4: Architecture Comparison `[Independent]`

The lesson concluded with the claim that Z-Image takes the same building blocks as SD3/Flux and recombines them more efficiently. This exercise tests that claim by comparing Z-Image directly against SD3 (or Flux if available).

### Your Task

1. **Load SD3 Medium** (or reuse from a previous notebook) alongside the already-loaded Z-Image Turbo
2. **Compare architectures:** parameter counts (text encoder, denoiser, VAE, total), text encoder setup (triple vs single), transformer block structure (dual-stream vs single-stream), attention configuration
3. **Generate the same prompt with both models** and compare output quality
4. **Trace the full Z-Image pipeline end-to-end**, annotating each step with the lesson that covered the relevant concept

### Hints

- To load SD3 alongside Z-Image, you may need to use `enable_model_cpu_offload()` on one or both pipelines to fit in GPU memory
- SD3 pipeline: `StableDiffusion3Pipeline.from_pretrained('stabilityai/stable-diffusion-3-medium-diffusers', torch_dtype=torch.float16)`
- Z-Image transformer config: `pipe.transformer.config` has `num_layers`, `num_attention_heads`, `attention_head_dim`, `patch_size`
- Z-Image uses `FlowMatchEulerDiscreteScheduler` (check `type(pipe.scheduler).__name__`)
- Z-Image's VAE reuses the Flux VAE -- check `pipe.vae.config`
- For the pipeline trace, follow this structure:
  ```
  1. Prompt -> Qwen3-4B text encoder [seq_len, d_model]     (this lesson)
  2. Text embeddings -> refiner layers                       (this lesson: S3-DiT)
  3. Noisy latent -> patchify -> image tokens               (Diffusion Transformers)
  4. Concatenate text + image + visual semantic tokens      (this lesson: S3-DiT)
  5. N S3-DiT blocks with shared attention + adaLN          (this lesson)
  6. Split output -> image tokens                           (this lesson)
  7. Unpatchify -> latent                                   (Diffusion Transformers)
  8. Flow matching sampling step                            (Flow Matching)
  9. Repeat for 8 steps (Decoupled-DMD)                    (this lesson)
  10. VAE decode -> pixels                                  (From Pixels to Latents)
  ```

### What to Print

- A side-by-side parameter comparison table
- Key architectural differences (text encoders, block structure, conditioning)
- Generated images from both models on the same prompt
- The full Z-Image pipeline trace with lesson annotations

If you cannot load SD3 Medium due to GPU memory constraints, compare against the SD3 numbers from the lesson (or from your previous notebook) instead of loading the model.

In [ ]:
# ============================================================
# Exercise 4: Architecture Comparison (Independent)
# ============================================================
#
# Compare Z-Image's architecture against SD3 (or Flux).
# Trace the full Z-Image pipeline end-to-end.
#
# Your code here:



In [ ]:
# --- Architecture comparison table ---
#
# Print a side-by-side comparison of Z-Image vs SD3 (or Flux).
# Include: text encoders, denoiser architecture, parameter counts,
# block structure, conditioning mechanism, training objective.
#
# Your code here:



In [ ]:
# --- Generate the same prompt with both models ---
#
# If you can load SD3, generate the same prompt with both
# and display side-by-side. If not, generate with Z-Image only
# and compare against a description of typical SD3 output.
#
# Your code here:



In [ ]:
# --- Full Z-Image pipeline trace ---
#
# Trace each stage of the Z-Image pipeline, printing shapes
# and annotating which lesson covered each concept.
#
# Your code here:



<details>
<summary>Solution</summary>

The comparison makes concrete what the lesson taught: Z-Image uses the same building blocks as SD3/Flux but combines them more efficiently. Every component traces to a lesson you completed.

**Architecture comparison (using Z-Image's actual config + SD3 numbers from the lesson):**

```python
# ============================================================
# Part 1: Architecture comparison table
# ============================================================

# Z-Image config from the loaded pipeline
zi_config = pipe.transformer.config
zi_transformer_params = count_parameters(pipe.transformer)
zi_encoder_params = count_parameters(pipe.text_encoder)
zi_vae_params = count_parameters(pipe.vae)
zi_total = zi_transformer_params + zi_encoder_params + zi_vae_params

print('Architecture Comparison: Z-Image vs SD3')
print('=' * 70)
print(f'{"Component":<30} {"SD3 Medium":<20} {"Z-Image Turbo":<20}')
print('-' * 70)
print(f'{"Text encoders":<30} {"3 (CLIP+OpenCLIP+T5)":<20} {"1 (Qwen3-4B)":<20}')
print(f'{"Text encoder params":<30} {"~5.2B":<20} {f"~{zi_encoder_params/1e9:.1f}B":<20}')
print(f'{"Denoiser type":<30} {"MMDiT (dual-stream)":<20} {"S3-DiT (single-stream)":<20}')
print(f'{"Denoiser params":<30} {"~2B":<20} {f"~{zi_transformer_params/1e9:.1f}B":<20}')
print(f'{"Total params":<30} {"~7.2B":<20} {f"~{zi_total/1e9:.1f}B":<20}')
print(f'{"Num layers":<30} {"24":<20} {f"{getattr(zi_config, \"num_layers\", \"?\")!s}":<20}')
print(f'{"Attention heads":<30} {"24":<20} {f"{getattr(zi_config, \"num_attention_heads\", \"?\")!s}":<20}')
print(f'{"Block structure":<30} {"Separate Q/K/V+FFN":<20} {"Shared Q/K/V+FFN":<20}')
print(f'{"Position encoding":<30} {"Learned":<20} {"3D Unified RoPE":<20}')
print(f'{"Training objective":<30} {"Flow matching":<20} {"Flow matching":<20}')
print(f'{"Turbo steps":<30} {"N/A":<20} {"8 steps":<20}')
print(f'{"Guidance (Turbo)":<30} {"N/A":<20} {"0.0 (baked in)":<20}')
print()
print('Key differences:')
print('  1. Text encoding: 3 specialized encoders -> 1 LLM')
print('  2. Block structure: dual-stream (per-modality Q/K/V+FFN) -> single-stream (shared)')
print('  3. Position: learned embeddings -> 3D Unified RoPE')
print('  4. Distillation: none -> Decoupled-DMD + DMDR')
print()
print('What is PRESERVED:')
print('  - Latent space generation (VAE encode/decode)')
print('  - Iterative denoising loop')
print('  - Flow matching training objective')
print('  - Transformer-based denoiser with patchify')
print('  - Pipeline structure: text encode -> denoise -> VAE decode')
```

**Generate with both (memory-efficient approach):**

```python
# Generate with Z-Image Turbo (already loaded)
prompt = 'a red ball to the left of a blue cube on a wooden table'
generator = torch.Generator(device='cuda').manual_seed(42)

zi_image = pipe(
    prompt=prompt,
    height=1024,
    width=1024,
    num_inference_steps=9,
    guidance_scale=0.0,
    generator=generator,
).images[0]

# If GPU memory allows, load SD3 as well.
# Otherwise, just display the Z-Image result.
try:
    from diffusers import StableDiffusion3Pipeline
    sd3_pipe = StableDiffusion3Pipeline.from_pretrained(
        'stabilityai/stable-diffusion-3-medium-diffusers',
        torch_dtype=torch.float16,
    )
    sd3_pipe.enable_model_cpu_offload()

    generator = torch.Generator(device='cpu').manual_seed(42)
    sd3_image = sd3_pipe(
        prompt=prompt,
        height=512, width=512,  # Lower res for memory
        num_inference_steps=28,
        guidance_scale=7.0,
        generator=generator,
    ).images[0]

    show_image_row(
        [sd3_image, zi_image],
        ['SD3 Medium\n(28 steps, ~7.2B params)',
         'Z-Image Turbo\n(8 steps, ~6B params)'],
        suptitle=f'"{prompt}"',
        figsize=(14, 6),
    )
    free_memory(sd3_pipe)
except Exception as e:
    print(f'Could not load SD3 (likely GPU memory): {e}')
    show_image_row(
        [zi_image],
        ['Z-Image Turbo\n(8 steps)'],
        suptitle=f'"{prompt}"',
        figsize=(8, 6),
    )
```

**Full Z-Image pipeline trace:**

```python
prompt = 'a cat sitting on a beach at sunset'

print('Z-Image Pipeline Trace')
print('=' * 70)
print()

# Get config values
config = pipe.transformer.config
scheduler_name = type(pipe.scheduler).__name__
vae_scale = pipe.vae_scale_factor
height, width = 1024, 1024
latent_h = height // vae_scale
latent_w = width // vae_scale
latent_c = getattr(config, 'in_channels', 16)
patch_size = getattr(config, 'patch_size', 2)
n_patches = (latent_h // patch_size) * (latent_w // patch_size)
n_layers = getattr(config, 'num_layers', 30)
d_model_val = getattr(config, 'joint_attention_dim', None)
if d_model_val is None:
    n_heads = config.num_attention_heads
    head_dim = config.attention_head_dim
    d_model_val = n_heads * head_dim

print('STAGE 1: TEXT ENCODING')
print('-' * 70)
print(f'  1. Prompt -> Qwen3-4B text encoder [seq_len, {d_model_val}]')
print(f'     One LLM replaces three encoders (CLIP + OpenCLIP + T5-XXL)')
print(f'     -> Lesson: Z-Image (this lesson), CLIP (6.3.3), SD3 (7.4.3)')
print()

print('STAGE 2: MODALITY PRE-PROCESSING (REFINER LAYERS)')
print('-' * 70)
print(f'  2. Text embeddings -> refiner layers -> aligned text tokens')
print(f'     Image patch embeddings -> refiner layers -> aligned image tokens')
print(f'     "Translate once, then speak together"')
print(f'     -> Lesson: Z-Image (this lesson: S3-DiT design)')
print()

print('STAGE 3: PATCHIFY')
print('-' * 70)
print(f'  3. Noisy latent [{latent_c}, {latent_h}, {latent_w}] -> patchify -> [{n_patches}, {d_model_val}]')
print(f'     Patch size: {patch_size}')
print(f'     -> Lesson: Diffusion Transformers (7.4.2)')
print()

print('STAGE 4: CONCATENATE (UNIFIED SEQUENCE)')
print('-' * 70)
print(f'  4. Concatenate: text tokens + image tokens + visual semantic tokens')
print(f'     All three token types in one unified sequence')
print(f'     -> Lesson: Z-Image (this lesson: S3-DiT)')
print()

print('STAGE 5: S3-DiT BLOCKS')
print('-' * 70)
print(f'  5. {n_layers} S3-DiT blocks: shared attention + shared FFN + adaLN')
print(f'     Single-stream: one set of Q/K/V, one SwiGLU FFN')
print(f'     Position: 3D Unified RoPE (temporal + spatial)')
print(f'     -> Lesson: Z-Image (this lesson) + DiT (7.4.2) + RoPE (4.2.3)')
print()

print('STAGE 6-7: SPLIT AND UNPATCHIFY')
print('-' * 70)
print(f'  6. Split: extract image tokens from unified sequence')
print(f'  7. Unpatchify: [{n_patches}, {d_model_val}] -> [{latent_c}, {latent_h}, {latent_w}]')
print(f'     -> Lesson: Diffusion Transformers (7.4.2)')
print()

print('STAGE 8-9: FLOW MATCHING SAMPLING')
print('-' * 70)
print(f'  8. Flow matching sampling step (scheduler: {scheduler_name})')
print(f'  9. Repeat for 8 steps (Decoupled-DMD distillation)')
print(f'     -> Lesson: Flow Matching (7.2.2) + Z-Image (this lesson)')
print()

print('STAGE 10: VAE DECODE')
print('-' * 70)
print(f' 10. VAE decode: [{latent_c}, {latent_h}, {latent_w}] -> [3, {height}, {width}]')
print(f'     Reuses the Flux VAE -- same proven reconstruction quality')
print(f'     -> Lesson: From Pixels to Latents (6.3.5)')
print()
print('=' * 70)
print('Every step traces to a lesson you completed.')
print('Nothing in this pipeline is unexplained.')
print('The code IS your knowledge, implemented.')
```

**Key observations:**
- The pipeline structure is preserved from SD3 to Z-Image: text encode, denoise in latent space, VAE decode. The components inside each stage evolved, but the pipeline did not.
- Z-Image's innovations are in simplification (single-stream, one encoder) and post-training (Decoupled-DMD, DMDR), not in new pipeline stages.
- Every component traces to a specific lesson: transformers (Series 4), RoPE (4.2.3), DiT/patchify (7.4.2), flow matching (7.2.2), latent diffusion (6.3.5), CLIP and text encoding (6.3.3), and the new concepts from this lesson.
- The comparison between Z-Image and SD3 on compositional prompts is particularly revealing: Z-Image's Qwen3-4B text encoder often handles spatial relationships and counting better than SD3's triple encoder setup, despite using fewer parameters.

**Common mistakes:**
- Not accounting for GPU memory when loading both SD3 and Z-Image. Use `enable_model_cpu_offload()` or compare against lesson numbers instead.
- Forgetting that Z-Image Turbo uses `guidance_scale=0.0` while SD3 uses `guidance_scale=7.0`. These are fundamentally different inference configurations.
- Conflating parameter count with quality. Z-Image achieves comparable quality with 1/5 the parameters, but the quality comes from training innovation (three-phase curriculum, Decoupled-DMD, DMDR), not just architecture.

</details>

---

## Key Takeaways

1. **"Translate once, then speak together."** Z-Image's S3-DiT uses shared Q/K/V projections and shared FFN across all token types. Lightweight refiner layers handle modality-specific pre-processing upfront, eliminating the ~50% parameter overhead of MMDiT's dual-stream design. You verified this by inspecting the actual transformer blocks.

2. **The text encoder trajectory completes.** CLIP (1 encoder) -> dual CLIP (2) -> CLIP + OpenCLIP + T5-XXL (3) -> Qwen3-4B (1 powerful LLM). Z-Image replaces three specialized encoders with one LLM. Simpler pipeline, richer embeddings, native bilingual support.

3. **Decoupled-DMD delivers at 8 steps.** Z-Image Turbo achieves high quality at 8 steps with `guidance_scale=0.0`. The distillation baked in the quality boost -- no CFG needed at inference. Sub-second on enterprise GPUs, under 16GB VRAM.

4. **Same building blocks, better combination.** Every component of Z-Image traces to a lesson you completed: transformers (Series 4), RoPE (4.2.3), patchify and adaLN (7.4.2), flow matching (7.2.2), latent diffusion (6.3.5). The innovation is in how they are combined: simpler architecture + better training + RL post-training.

5. **Simplicity beats complexity.** Z-Image matches 32B Flux with ~6B parameters. Not through architectural novelty but through eliminating unnecessary duplication and investing in better training. The architecture is simpler than MMDiT. The papers are not beyond your understanding -- they ARE your understanding, combined.